In [2]:
import numpy as np
# import matplotlib.pyplot as plt

In [3]:
H = np.loadtxt("Hmatrix3.txt",delimiter=',')
Axes0 = H.shape[0] #Rows
Axes1 = H.shape[1] #Cols
print(Axes0,Axes1)
# print(H)

9 12


In [4]:
class Node:
    def __init__(self, nodeID, connected_nodeList, bitValue,*args,**kwargs): 
        self.nodeID = nodeID #Helps to locate node in general list
        self.adjNodes = connected_nodeList # List of connected nodes to a particular node
        self.prevMsg = 2 #Set 2 as there is no initial messsage when started
        self.currMsg = bitValue
        self.pr1 = 0.5 if bitValue==-1 else bitValue #Stores probab=0.5 for erasure else same as bit.
        self.FrmAdjNode = [None]*len(connected_nodeList) #Msg received from adj nodes

# Pi = Pr(ci = 1|yi)
# qij is a message sent by the variable node to the check node
# Every message contains always the pair qij(0) and qij(1) which stands for the amount of belief that yi is a '0' or a '1'.
# rji is a message sent by the check node fj to the variable node. Again there is a rji(0) and rji(1) that indicates the (current) amount of believe in that yi is a '0' or a '1'.
    def varToChk(self,chkNodes):
        #For 1st time sends whatever probability it has otherwise it makes a decision based on what it received from other check nodes (except the one it is sending to). 
        # They use this equation to update their response messages:
        # qij(0) = Kij*(1-Pi)*PI[rj'i(0)] where PI is multiplication of all values except that of the checknode it is currently sending response to.
        #K must be determined in such a way that qij(0) + qij(1) = 1.
        # Then, it considers decisions of all checknodes to update is currMsg using hard deicision.
        if self.prevMsg==2:
            for nodeInd in self.adjNodes: # From variable nodes adjacent node list 
                idInd = chkNodes[nodeInd].adjNodes.index(self.nodeID) #Index of checknodes in chkNodes list
                chkNodes[nodeInd].FrmAdjNode[idInd] = self.pr1
            self.prevMsg = self.currMsg
        else:
            self.prevMsg = self.currMsg
            for chkNodeInd in self.adjNodes:
                pr1 = self.pr1
                pr0 = 1 - pr1
                for prob1 in self.FrmAdjNode: #Iterate through probab list sent by chk nodes
                    if  self.FrmAdjNode.index(prob1) == self.adjNodes.index(chkNodeInd):
                        continue
                    else:
                        pr1 = pr1*prob1
                        pr0 = pr0*(1 - prob1)

                k = 1/(pr0 + pr1)
                
                varNodeInd = chkNodes[chkNodeInd].adjNodes.index(self.nodeID)
                chkNodes[chkNodeInd].FrmAdjNode[varNodeInd] = k*pr1

            new_pr1 = pr1*self.FrmAdjNode[-1] #After last iteration pr1 will have multiplication of all probs except last element
            new_pr0 = pr0*(1 - self.FrmAdjNode[-1])
            
            #Generating updated belief and hard decision
            # new_pr1 = self.pr1
            # new_pr0 = 1 - self.pr1
            # for prob in self.FrmAdjNode:
            #     new_pr1*=prob
            #     new_pr0*=(1-prob)
            
            # if(new_pr0 == 0 and new_pr1==0):
            #     new_pr0 = new_pr1 = 0.000001
            # k_1 = 1/(new_pr0 + new_pr1)
            # # # print("M2: ",new_pr0,new_pr1)
            # new_pr0*=k_1
            # new_pr1*=k_1
            # print(self.nodeID, " P(1): ",new_pr1," P(0): ",new_pr0)
            # if self.currMsg==-1:
            #     self.pr1 = new_pr1
            if(new_pr1 > new_pr0):
                self.currMsg = 1
            elif (new_pr0 > new_pr1):
                self.currMsg = 0
            else:
                self.currMsg = -1   
                

    def chkToVar(self,varNodes):
        #check node considers message from variable nodes and calculates response for particular variable node using equation rji(0) = 0.5 + 0.5*PI[(1-2*q_i'j(1))] where PI is multiplication of all messsage values expect for the one variable node to which response needs to be sent. (This is parity check equation.) 
        for varNodeInd in self.adjNodes:
            prob0 = 0.5
            for pr1 in self.FrmAdjNode: #Iterate through probab list sent by var nodes
                if self.FrmAdjNode.index(pr1) == self.adjNodes.index(varNodeInd):
                    continue
                else:
                    prob0 = prob0*(1-2*pr1)

            chknodeInd = varNodes[varNodeInd].adjNodes.index(self.nodeID)
            varNodes[varNodeInd].FrmAdjNode[chknodeInd] = 0.5 - prob0 #1-(0.5+0.5*(1-2qi'j))


In [276]:
def soft_decode_bec(varNodes,chkNodes):
    t = 0 #Iteration index
    while(t<50):

        #variable node to check node logic
        for varNode in varNodes:
            varNode.varToChk(chkNodes)

        #check node to variable node logic
        for chkNode in chkNodes:
            chkNode.chkToVar(varNodes)
            
                
        #Loop break condition and error counting
        currWord = []
        for parity in varNodes:
            currWord.append(parity.currMsg)

        
        if -1 not in currWord:
            # print(currWord)
            if sum(currWord)%2 == 0:
                return currWord,t

            
        t += 1
    return currWord,t

In [277]:
def generate_msg(p):
    # p = 0.3 #Probability of a bit to be an erasure
    Msg = np.random.randint(0, 1, size=Axes1)
    receivedMsg = Msg.copy()
    receivedMsg[np.random.rand(Axes1) < p] = -1
    # print(receivedMsg)
    return list(Msg),list(receivedMsg)

In [278]:
def initNodes(receivedMsg):
    varNodes = []
    chkNodes = []
    for i in range(Axes0): 
        temp1 = list(H[i].nonzero()[0]) #Finding adj nodes
        chkNodes.append(Node(nodeID=i, connected_nodeList=temp1, bitValue=2)) #Checknode initially has no message

    for i in range(Axes1): 
        temp2 = list(H[:,i].nonzero()[0])
        varNodes.append(Node(nodeID=i, connected_nodeList=temp2, bitValue=receivedMsg[i])) #Initializing variable nodes with received message
    
    return varNodes,chkNodes


In [279]:
L = [0, -1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0] #Error case

originalMsg,errorMsg = generate_msg(1)
varNodes,chkNodes = initNodes(receivedMsg=errorMsg)
decoded_wrd,iter = soft_decode_bec(varNodes,chkNodes)
print(errorMsg,decoded_wrd,iter)


[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1] [0, 0, 0, 0, 0, 0, 0, -1, 0, 0, -1, -1] 50
